In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.patches as patches
import random
import numpy as np
import matplotlib.pyplot as plt
import pathlib

from scipy.stats import linregress
from scipy.stats import chi2_contingency, spearmanr, ttest_ind

In [ ]:
path_info_opp = pd.read_csv('info_iid_opposite_28s.csv',sep =";", header=0)
path_info_opp['acc'] =path_info_opp['correct'].apply(lambda x: x/28 )

In [ ]:
path_info_same = pd.read_csv('info_iid_same_28s.csv',sep =";", header=0)
path_info_same['acc'] =path_info_same['correct'].apply(lambda x: x/28 )

In [ ]:
def path_responces_exp1 (path_info):
    
    path_info = path_info.drop(['ok', "more corr answ", "mean=last10"], axis = 1)
    path_info['mean/std'] = round(abs(abs(path_info['Mean'])/ path_info['Standard Deviation']),3)
    path_info['acc'] = path_info['correct'].apply(lambda x: np.round( x/28,3) )
    group_sd = []
    intervals_sd = create_intervals(path_info['Standard Deviation'].min(), path_info['Standard Deviation'].max(), 
                                    path_info['Standard Deviation'].mean())
    print(intervals_sd)
    for i in range(path_info.shape[0]):
        if path_info['Standard Deviation'][i] < 3.3:
            group_sd.append(1)
        if 3.3<= path_info['Standard Deviation'][i] < 5.5:
            group_sd.append(2) 
    #     if 4.42 <= df['sd'][i] < 5.7:
    #         group_sd.append(3)     
        if path_info['Standard Deviation'][i] >=5.5:
            group_sd.append(3)  
    path_info["group_sd"] = group_sd
    
    diff_list = []

    jnd_values = {1: 0.62, 2: 0.89, 3: 1.27}
    # jnd_values = {1: 1.34, 2: 1.30, 3: 2.26}
    for i in range(path_info.shape[0]):
        group = path_info['group_sd'][i]
        jnd = jnd_values[group]
    #     diff = round(abs(iid_opposite_data['mean_abs'][i] - jnd), 3)
        diff = round(abs(path_info['Mean'][i]) - jnd, 3)
        diff_list.append(diff)

    path_info['mu-jnd'] = diff_list
    return path_info


def create_intervals(min_value, max_value, mean_value):
    # Small interval
    small_interval = [min_value, round(mean_value - (mean_value - min_value) / 2,3)]
    # Mean interval
    mean_interval = [round(mean_value - (mean_value - min_value) / 2,3), round(mean_value + (max_value - mean_value) / 2,3)]
    # Big interval
    big_interval = [round(mean_value + (max_value - mean_value) / 2,3), max_value]
    return small_interval, mean_interval, big_interval

In [ ]:
path_info_opp = path_responces_exp1(path_info_opp)

In [ ]:
path_info_same = path_responces_exp1(path_info_same)

In [ ]:
### subplot1 ###

# Setup the plot with large font size for publication
plt.figure(figsize=(8, 6))
plt.rcParams.update({'font.size': 20, 'font.family': 'Times New Roman'})

# Colors in grayscale
colors = ['black', 'gray', 'silver']  # Different shades of gray for differentiation



# Opposite
slope, intercept, r_value, p_value, std_err = linregress(path_info_opp['mu-jnd'], path_info_opp['acc'])
x_vals = np.array(plt.gca().get_xlim())
x_vals = np.array([-0.5,1.7])
y_vals = intercept + slope * x_vals
plt.plot(x_vals, y_vals, '-', color=colors[0], label=f'IID Opposite: y={slope:.2f}x+{intercept:.2f}, $R^2$={r_value**2:.2f}')
plt.scatter(path_info_opp['mu-jnd'], path_info_opp['acc'], alpha=0.6, c=colors[0],)

slope, intercept, r_value, p_value, std_err = linregress(path_info_same['mu-jnd'], path_info_same['acc'])
# x_vals = np.array(plt.gca().get_xlim())
x_vals = np.array([-0.5,1.7])
y_vals = intercept + slope * x_vals
plt.plot(x_vals, y_vals, '-', color=colors[1], label=f'IID same: y={slope:.2f}x+{intercept:.2f}, $R^2$={r_value**2:.2f}')
plt.scatter(path_info_same['mu-jnd'], path_info_same['acc'], alpha=0.6, c=colors[1],)



plt.xlabel('mean - JND ')
plt.ylabel('Accuracy')
plt.ylim(-0.0, 1.1)
plt.legend(loc='lower right', fontsize=16)
plt.show()

In [ ]:
iid_equal = pd.read_csv('Exp_2\\2_Script\\all_subject_equal.csv',sep ="\t", header=0)
iid_n_equal = pd.read_csv('Exp_2\\2_Script\\all_subject_n_equal.csv',sep ="\t", header=0)

In [ ]:
def path_responces (path):
    grouped_data = path.groupby(['index_traj', 'condition', 'mean_side']).agg(
    path=pd.NamedAgg(column='path', aggfunc='first'),
    last_cond=pd.NamedAgg(column='last_cond', aggfunc='first'),
    index_old=pd.NamedAgg(column='index_old', aggfunc='first'),
    mean_val=pd.NamedAgg(column='mean_val', aggfunc='first'),
    mean_last5=pd.NamedAgg(column='mean_last5', aggfunc='first'),
    mean_last15=pd.NamedAgg(column='mean_last15', aggfunc='first'),
    sd=pd.NamedAgg(column='sd', aggfunc='first'),
    last_visibl_side=pd.NamedAgg(column='last_visibl_side', aggfunc='first'),
    f_missing_pos=pd.NamedAgg(column='f_missing_pos', aggfunc='first'),
    last_visibl_center_dist_deg=pd.NamedAgg(column='last_visibl_center_dist[deg]', aggfunc='first'),
    correct=pd.NamedAgg(column='accuracy', aggfunc=lambda x: (x == 1).sum()),
    error=pd.NamedAgg(column='accuracy', aggfunc=lambda x: (x == 0).sum()),
    acc=pd.NamedAgg(column='accuracy', aggfunc=lambda x: np.round(x.mean(), 3)),
    RT=pd.NamedAgg(column='RT', aggfunc=lambda x: np.round(x.mean(), 3))).reset_index()
    grouped_data['mean_abs'] = grouped_data['mean_val'].apply(abs)
    group_sd = []
    for sd in grouped_data['sd']:
        if sd < 3.4:
            group_sd.append(1)
        elif 3.4 <= sd < 5.3:
            group_sd.append(2)
        else:  # Assuming sd >= 5.5
            group_sd.append(3)
    grouped_data['group_sd'] = group_sd
    grouped_data['mean/std'] = round( abs(grouped_data['mean_val']/ grouped_data['sd']),3)
    # Calculating the difference between mean_abs and JND for each group
    diff_list = []
    # jnd_values = {1: 0.386, 2: 0.581, 3: 0.654}
    # jnd_values = {1: 1.3, 2: 1.3, 3: 1.3}
    jnd_values = {1: 0.6, 2: 0.9, 3: 1.3}
    for i in range(grouped_data.shape[0]):
        group = grouped_data['group_sd'][i]
        jnd = jnd_values[group]
    #     diff = round(abs(grouped_data['mean_abs'][i] - jnd), 3)
        diff = round(grouped_data['mean_abs'][i] - jnd, 3)
        diff_list.append(diff)

    grouped_data['mu-jnd'] = diff_list

    return grouped_data
    

In [ ]:
grouped_iid_n_equal=path_responces (iid_n_equal)
grouped_iid_equal=path_responces (iid_equal)

In [ ]:
# Setup the plot with large font size for publication
plt.figure(figsize=(8, 6))
plt.rcParams.update({'font.size': 20, 'font.family': 'Times New Roman'})

# Colors in grayscale
colors = ['black', 'gray', 'silver']  # Different shades of gray for differentiation

# Scatter plot by group
groups = grouped_iid_equal['path'].unique()
for i, group in enumerate(groups):
    subset = grouped_iid_equal[grouped_iid_equal['path'] == group]
    plt.scatter(subset['mu-jnd'], subset['acc'], alpha=0.5, c=colors[i],)

# Overall regression
slope, intercept, r_value, p_value, std_err = linregress(grouped_iid_equal['mu-jnd'], grouped_iid_equal['acc'])
x_vals = np.array(plt.gca().get_xlim())
y_vals = intercept + slope * x_vals
# plt.plot(x_vals, y_vals, '--', color='black', label=f'Overall: y={slope:.2f}x+{intercept:.2f}, $R^2$={r_value**2:.2f}')

# Regression by group
for i, group in enumerate(groups):
    group_data = grouped_iid_equal[grouped_iid_equal['path'] == group]
    slope, intercept, r_value, p_value, std_err = linregress(group_data['mu-jnd'], group_data['acc'])
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, '-', color=colors[i], label=f'Aligned "last {group}": y={slope:.2f}x+{intercept:.2f}, $R^2$={r_value**2:.2f}')

plt.xlabel('mean - JND ')
plt.ylabel('Accuracy')
plt.ylim(0, 1.1)
plt.legend(loc='lower right', fontsize=16)
plt.show()

In [ ]:
### subplot1 ###

# Setup the plot with large font size for publication
plt.figure(figsize=(8, 6))
plt.rcParams.update({'font.size': 20, 'font.family': 'Times New Roman'})

# Colors in grayscale
colors = ['black', 'gray', 'silver']  # Different shades of gray for differentiation



# Opposite
slope, intercept, r_value, p_value, std_err = linregress(path_info_opp['mu-jnd'], path_info_opp['acc'])
x_vals = np.array(plt.gca().get_xlim())
x_vals = np.array([-0.5,1.7])
y_vals = intercept + slope * x_vals
plt.plot(x_vals, y_vals, '-', color=colors[0], label=f'IID Opposite: y={slope:.2f}x+{intercept:.2f}, $R^2$={r_value**2:.2f}')
plt.scatter(path_info_opp['mu-jnd'], path_info_opp['acc'], alpha=0.6, c=colors[0],)

slope, intercept, r_value, p_value, std_err = linregress(path_info_same['mu-jnd'], path_info_same['acc'])
x_vals = np.array(plt.gca().get_xlim())
x_vals = np.array([-0.5,1.7])
y_vals = intercept + slope * x_vals
plt.plot(x_vals, y_vals, '-', color=colors[1], label=f'IID same: y={slope:.2f}x+{intercept:.2f}, $R^2$={r_value**2:.2f}')
plt.scatter(path_info_same['mu-jnd'], path_info_same['acc'], alpha=0.6, c=colors[1],)



plt.xlabel('mean - JND ')
plt.ylabel('Accuracy')
plt.ylim(-0.0, 1.1)
plt.legend(loc='lower right', fontsize=16)
plt.show()


#### subplot 2###


# Setup the plot with large font size for publication
plt.figure(figsize=(8, 6))
plt.rcParams.update({'font.size': 20, 'font.family': 'Times New Roman'})

# Colors in grayscale
colors = ['black', 'gray', 'silver']  # Different shades of gray for differentiation

# Scatter plot by group
groups = grouped_iid_n_equal['path'].unique()
for i, group in enumerate(groups):
    subset = grouped_iid_n_equal[grouped_iid_n_equal['path'] == group]
    plt.scatter(subset['mu-jnd'], subset['acc'], alpha=0.5, c=colors[i],)

# Overall regression
slope, intercept, r_value, p_value, std_err = linregress(grouped_iid_n_equal['mu-jnd'], grouped_iid_n_equal['acc'])
x_vals = np.array(plt.gca().get_xlim())
y_vals = intercept + slope * x_vals
# plt.plot(x_vals, y_vals, '--', color='black', label=f'Overall: y={slope:.2f}x+{intercept:.2f}, $R^2$={r_value**2:.2f}')

# Regression by group
for i, group in enumerate(groups):
    group_data = grouped_iid_n_equal[grouped_iid_n_equal['path'] == group]
    slope, intercept, r_value, p_value, std_err = linregress(group_data['mu-jnd'], group_data['acc'])
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, '-', color=colors[i], label=f'Misaligned "last {group}": y={slope:.2f}x+{intercept:.2f}, $R^2$={r_value**2:.2f}')

plt.xlabel('mean - JND ')
plt.ylabel('Accuracy')
plt.ylim(0, 1.1)
plt.legend(loc='lower right', fontsize=16)
plt.show()

## subplot 3

# Setup the plot with large font size for publication
plt.figure(figsize=(8, 6))
plt.rcParams.update({'font.size': 20, 'font.family': 'Times New Roman'})

# Colors in grayscale
colors = ['black', 'gray', 'silver']  # Different shades of gray for differentiation

# Scatter plot by group
groups = grouped_iid_equal['path'].unique()
for i, group in enumerate(groups):
    subset = grouped_iid_equal[grouped_iid_equal['path'] == group]
    plt.scatter(subset['mu-jnd'], subset['acc'], alpha=0.5, c=colors[i],)

# Overall regression
slope, intercept, r_value, p_value, std_err = linregress(grouped_iid_equal['mu-jnd'], grouped_iid_equal['acc'])
x_vals = np.array(plt.gca().get_xlim())
y_vals = intercept + slope * x_vals
# plt.plot(x_vals, y_vals, '--', color='black', label=f'Overall: y={slope:.2f}x+{intercept:.2f}, $R^2$={r_value**2:.2f}')

# Regression by group
for i, group in enumerate(groups):
    group_data = grouped_iid_equal[grouped_iid_equal['path'] == group]
    slope, intercept, r_value, p_value, std_err = linregress(group_data['mu-jnd'], group_data['acc'])
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, '-', color=colors[i], label=f'Aligned "last {group}": y={slope:.2f}x+{intercept:.2f}, $R^2$={r_value**2:.2f}')

plt.xlabel('mean - JND ')
plt.ylabel('Accuracy')
plt.ylim(0, 1.1)
plt.legend(loc='lower right', fontsize=16)
plt.show()

In [ ]:
groups = grouped_iid_n_equal['path'].unique()
for i, group in enumerate(groups):
    subset = grouped_iid_n_equal[grouped_iid_n_equal['path'] == group]
    plt.scatter(subset['mean/std'], subset['acc'], alpha=0.5, c=colors[i],)

# Overall regression
# slope, intercept, r_value, p_value, std_err = linregress(grouped_iid_n_equal['mean/std'], grouped_iid_n_equal['acc'])
# x_vals = np.array(plt.gca().get_xlim())
# y_vals = intercept + slope * x_vals
# plt.plot(x_vals, y_vals, '--', color='black', label=f'Overall: y={slope:.2f}x+{intercept:.2f}, $R^2$={r_value**2:.2f}')

# Regression by group
for i, group in enumerate(groups):
    group_data = grouped_iid_n_equal[grouped_iid_n_equal['path'] == group]
    slope, intercept, r_value, p_value, std_err = linregress(group_data['mean/std'], group_data['acc'])
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, '-', color=colors[i], label=f'Misaligned "last {group}": y={slope:.2f}x+{intercept:.2f}, $R^2$={r_value**2:.2f}')

plt.xlabel('mean - JND ')
plt.ylabel('Accuracy')
plt.ylim(0, 1.1)
plt.legend(loc='lower right', fontsize=16)
plt.show()

In [ ]:
# Setup the plot with large font size for publication
plt.figure(figsize=(20, 10))
plt.rcParams.update({'font.size': 25, 'font.family': 'Times New Roman'})

# Colors in grayscale
colors = ['black', 'gray', 'silver']  # Different shades of gray for differentiation

# Subplot 1
plt.subplot(1, 2, 1)
slope, intercept, r_value, p_value, std_err = linregress(path_info_opp['mu-jnd'], path_info_opp['acc'])
x_vals = np.array([-0.5, 1.7])
y_vals = intercept + slope * x_vals
plt.plot(x_vals, y_vals, '-', color=colors[0], label=f'IID Opposite: y={slope:.2f}x+{intercept:.2f}, $R^2$={r_value**2:.2f}',linewidth=3)
plt.scatter(path_info_opp['mu-jnd'], path_info_opp['acc'], alpha=0.6, c=colors[0],s=80)

slope, intercept, r_value, p_value, std_err = linregress(path_info_same['mu-jnd'], path_info_same['acc'])
y_vals = intercept + slope * x_vals
plt.plot(x_vals, y_vals, '-', color=colors[1], label=f'IID Same: y={slope:.2f}x+{intercept:.2f}, $R^2$={r_value**2:.2f}',linewidth=3)
plt.scatter(path_info_same['mu-jnd'], path_info_same['acc'], alpha=0.6, c=colors[1],s=80)

plt.xlabel('Mean - JND')
plt.ylabel('Accuracy')
plt.ylim(-0.0, 1.1)
plt.legend(loc='lower right', fontsize=22)

# Subplot 2
plt.subplot(1, 2, 2)
groups = grouped_iid_n_equal['path'].unique()
for i, group in enumerate(groups):
    subset = grouped_iid_n_equal[grouped_iid_n_equal['path'] == group]
    plt.scatter(subset['mu-jnd'], subset['acc'], alpha=0.5, c=colors[i],s=80)
    
for i, group in enumerate(groups):
    # x_vals = np.array(plt.gca().get_xlim())
    group_data = grouped_iid_n_equal[grouped_iid_n_equal['path'] == group]
    slope, intercept, r_value, p_value, std_err = linregress(group_data['mu-jnd'], group_data['acc'])
    y_vals = intercept + slope * x_vals
    slope1, intercept1, r_value1, p_value1, std_err1 = linregress(group_data['mean/std'], group_data['acc'])
    plt.plot(x_vals, y_vals, '-', color=colors[i], label=f'Misaligned "last {group}": y={slope1:.2f}x+{intercept1:.2f}, $R^2$={r_value1**2:.2f}',linewidth=3)

plt.xlabel('Mean - JND')
plt.ylabel('Accuracy')
plt.ylim(0, 1.1)
plt.legend(loc='lower right', fontsize=22)


plt.tight_layout()
plt.show()